In [2]:
import requests
import pandas as pd
import my_key as key
import csv
import json
import plotly.express as px

In [3]:
api_key = key.api_key

In [4]:
# Changer le chemin du fichier si nécessaire

df_gps_data = pd.read_csv("gps_data.csv")
df_gps_data.head()


,City,Latitude,Longitude
0,Mont Saint Michel,48.635954,-1.511460
1,Saint Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [5]:
# Récupère la température et la météo
def get_temperature_humidity_weather(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature = data["current"]["temp"] - 273.15  # Convertit la température en Celcius
        humidity = data["current"]["humidity"]
        weather = data["current"]["weather"][0]["main"]
        return temperature, humidity, weather
    else:
        return None, None, None

In [6]:
csv_filename = "meteo_data.csv"

# Créer un fichier CSV
with open('meteo_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['City', 'Temperature (°C)', 'Humidity (%)', 'Weather']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df_gps_data.iterrows():
        city = row['City']
        latitude = row['Latitude']
        longitude = row['Longitude']
        temperature, humidity, weather = get_temperature_humidity_weather(latitude, longitude)
        if temperature is not None:
            temperature = round(temperature)  # Arrondir la température à l'entier le plus proche
            writer.writerow({'City': city, 'Temperature (°C)': temperature, 'Humidity (%)': humidity, 'Weather': weather})
            print(f"City: {city}, Temperature : {temperature}°C, Humidity : {humidity}%, Weather: {weather}")
        else:
            writer.writerow({'City': city, 'Temperature (°C)': 'N/A', 'Humidity (%)': 'N/A', 'Weather': 'N/A'})

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

City: Mont Saint Michel, Temperature : 16°C, Humidity : 89%, Weather: Clouds
City: Saint Malo, Temperature : 16°C, Humidity : 88%, Weather: Clouds
City: Bayeux, Temperature : 15°C, Humidity : 91%, Weather: Clouds
City: Le Havre, Temperature : 16°C, Humidity : 100%, Weather: Mist
City: Rouen, Temperature : 16°C, Humidity : 86%, Weather: Clouds
City: Paris, Temperature : 16°C, Humidity : 81%, Weather: Clouds
City: Amiens, Temperature : 15°C, Humidity : 83%, Weather: Rain
City: Lille, Temperature : 15°C, Humidity : 86%, Weather: Rain
City: Strasbourg, Temperature : 16°C, Humidity : 67%, Weather: Clouds
City: Chateau du Haut Koenigsbourg, Temperature : 11°C, Humidity : 77%, Weather: Clear
City: Colmar, Temperature : 14°C, Humidity : 79%, Weather: Clear
City: Eguisheim, Temperature : 14°C, Humidity : 79%, Weather: Clear
City: Besancon, Temperature : 10°C, Humidity : 96%, Weather: Clouds
City: Dijon, Temperature : 14°C, Humidity : 77%, Weather: Clouds
City: Annecy, Temperature : 12°C, Humidi

In [7]:
df_meteo = pd.read_csv("meteo_data.csv")
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,16,89,Clouds
1,Saint Malo,16,88,Clouds
2,Bayeux,15,91,Clouds
3,Le Havre,16,100,Mist
4,Rouen,16,86,Clouds


In [8]:
df_meteo.rename(columns={'temperature (°C)': 'temperature', 'humidite (%)': 'humidite'}, inplace=True)
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,16,89,Clouds
1,Saint Malo,16,88,Clouds
2,Bayeux,15,91,Clouds
3,Le Havre,16,100,Mist
4,Rouen,16,86,Clouds


Création d'un seul fichier csv avec toute les données

In [9]:
with open('booking.json') as f:
    data_json = json.load(f)

In [10]:
# Modifier les noms de ville dans le JSON
for entry in data_json:
    if 'ville' in entry:
        entry['ville'] = entry['ville'].replace('+', ' ')

In [11]:
# Créer un DataFrame à partir du JSON
df_json = pd.DataFrame(data_json)
df_json.head()

,ville,name,website,score,desc,lat,lon
0,Aigues-Mortes%2C Languedoc-Roussillon%2C France,"appartement n2 centre village, clim, parking g...",https://www.booking.com/hotel/fr/appartement-n...,3 sur 5,Hébergement géré par un particulier,43.56724070,4.19380070
1,Aigues-Mortes%2C Languedoc-Roussillon%2C France,Cabane STELLA Aigues-Mortes,https://www.booking.com/hotel/fr/la-cabane-du-...,3 sur 5,Hébergement géré par un particulier,43.57930210,4.21283370
2,Aigues-Mortes%2C Languedoc-Roussillon%2C France,LE FLAMANT ROSE - COSYKAZ,https://www.booking.com/hotel/fr/le-flamant-ro...,3 sur 5,Hébergement géré par un particulier,43.57152370,4.17889910
3,Aigues-Mortes%2C Languedoc-Roussillon%2C France,B&B La Terre Brûlée,https://www.booking.com/hotel/fr/b-amp-b-la-te...,3 sur 5,"Situé à Aigues-Mortes, le B&B La Terre Brûlée ...",43.56197612,4.20767969
4,Aigues-Mortes%2C Languedoc-Roussillon%2C France,Au Cœur des Remparts,https://www.booking.com/hotel/fr/au-coeur-des-...,3 sur 5,Hébergement géré par un particulier,43.56540100,4.19297300


In [12]:
df_json.rename(columns={"ville" : "City"}, inplace=True)

In [13]:
df_fusionne = df_meteo.merge(df_json, on='City', how='left')
df_fusionne.head()

,City,Temperature (°C),Humidity (%),Weather,name,website,score,desc,lat,lon
0,Mont Saint Michel,16,89,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.61470049,-1.50961697
1,Mont Saint Michel,16,89,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.59775819,-1.51228422
2,Mont Saint Michel,16,89,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.61561170,-1.48851210
3,Mont Saint Michel,16,89,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.62259438,-1.39050267
4,Mont Saint Michel,16,89,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.59456300,-1.61772500


In [14]:
new_col_names = {
    'City': 'city',
    'Temperature (°C)': 'temperature',
    'Humidity (%)': 'humidity',
    'Weather': 'weather',
    'name': 'hotel',
    'website': 'website',
    'score': 'score',
    'desc': 'description',
    'lat': 'lat',
    'lon': 'lon',
}


In [15]:
df_fusionne = df_fusionne.rename(columns=new_col_names)
df_fusionne.head()

,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Mont Saint Michel,16,89,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.61470049,-1.50961697
1,Mont Saint Michel,16,89,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.59775819,-1.51228422
2,Mont Saint Michel,16,89,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.61561170,-1.48851210
3,Mont Saint Michel,16,89,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.62259438,-1.39050267
4,Mont Saint Michel,16,89,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.59456300,-1.61772500


In [16]:
df_final = df_fusionne[
    ['city', 'temperature', 'humidity', 'weather', 
     'hotel','website','score','description','lat','lon'
     ]
    ]


In [17]:
df_final.to_csv('data_final.csv', index=True, index_label='id')


In [18]:
df_fichier_final = pd.read_csv("data_final.csv")
df_fichier_final.head()

,id,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,0,Mont Saint Michel,16,89,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.614700,-1.509617
1,1,Mont Saint Michel,16,89,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.597758,-1.512284
2,2,Mont Saint Michel,16,89,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.615612,-1.488512
3,3,Mont Saint Michel,16,89,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.622594,-1.390503
4,4,Mont Saint Michel,16,89,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.594563,-1.617725


In [19]:
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,16,89,Clouds
1,Saint Malo,16,88,Clouds
2,Bayeux,15,91,Clouds
3,Le Havre,16,100,Mist
4,Rouen,16,86,Clouds


Top 5 weather localization

In [20]:
filtered_df = df_fichier_final[(df_fichier_final["weather"] == "Clear") | (df_fichier_final["weather"] == "light rain")]
filtered_df

,id,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
161,161,Chateau du Haut Koenigsbourg,11,77,Clear,Domaine du Haut-Koenigsbourg,https://www.booking.com/hotel/fr/les-chambres-...,3 sur 5,Le Domaine du Haut-Koenigsbourg est situé à Or...,48.247023,7.346321
162,162,Chateau du Haut Koenigsbourg,11,77,Clear,Hôtel Winzenberg,https://www.booking.com/hotel/fr/winzenberg.fr...,2 sur 5,L'Hôtel Winzenberg est situé dans le village v...,48.341664,7.418982
163,163,Chateau du Haut Koenigsbourg,11,77,Clear,Gîte MARTINS,https://www.booking.com/hotel/fr/gite-martins....,2 sur 5,Hébergement géré par un particulier,48.232325,7.300680
164,164,Chateau du Haut Koenigsbourg,11,77,Clear,Gite Au Nid De Cigognes,https://www.booking.com/hotel/fr/gite-au-nid-d...,2 sur 5,Hébergement géré par un particulier,48.309825,7.418319
165,165,Chateau du Haut Koenigsbourg,11,77,Clear,Les Apparts du Gewur'z,https://www.booking.com/hotel/fr/les-apparts-d...,2 sur 5,Hébergement géré par un particulier,48.205307,7.363375
...,...,...,...,...,...,...,...,...,...,...,...
597,597,Montauban,17,78,Clear,Hôtel du Commerce,https://www.booking.com/hotel/fr/hotel-du-comm...,3 sur 5,"Situé dans le centre de Montauban, l'Hôtel du ...",44.015426,1.354073
598,598,Montauban,17,78,Clear,Sure Hotel by Best Western Les Portes de Monta...,https://www.booking.com/hotel/fr/sure-by-best-...,3 sur 5,"Situé à Montauban, à 42 km de l'amphithéâtre P...",43.936287,1.322098
599,599,Montauban,17,78,Clear,L'Amarelo,https://www.booking.com/hotel/fr/amarelo.fr.ht...,2 sur 5,Hébergement géré par un particulier,44.021308,1.351224
600,600,Montauban,17,78,Clear,Cosy Red 4 Pers - Neuf et au Calme - Fibre-Net...,https://www.booking.com/hotel/fr/cosy-red.fr.h...,3 sur 5,"Situé à Montauban, en Midi-Pyrénées, le Cosy R...",44.013431,1.360972


In [21]:
# Best weather
filtered_df = df_fichier_final[(df_fichier_final["weather"] == "Clear") | (df_fichier_final["weather"] == "light rain")]

# Then group by 'city', select the first entry, sort by 'temperature', and take the top 5
df_top_five = filtered_df.groupby(by='city').first().sort_values(by=['temperature'], ascending=False).reset_index().head(10)

df_top_five.head()

,city,id,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Bormes les Mimosas,341,22,44,Clear,Hotel des Arts - Cite Bergere,https://www.booking.com/hotel/fr/des-arts-pari...,2 sur 5,L'Hotel des Arts est situé dans le centre de P...,48.871880,2.344655
1,Aigues Mortes,481,21,46,Clear,None,None,None,None,NaN,NaN
2,Collioure,502,21,62,Clear,Grand Hotel des Balcons,https://www.booking.com/hotel/fr/grand-des-bal...,2 sur 5,Situé dans le quartier chic de Saint-Germain à...,48.850422,2.339299
3,Cassis,361,20,42,Clear,LE SEPT charmant studio aux portes des calanques,https://www.booking.com/hotel/fr/le-sept-charm...,3 sur 5,Hébergement géré par un particulier,43.217600,5.532350
4,Aix en Provence,401,19,50,Clear,La Suite Albertas,https://www.booking.com/hotel/fr/le-555-mirabe...,3 sur 5,"Situé à Aix-en-Provence, à 100 mètres du cours...",43.527418,5.447695


In [22]:
csv_filename = "top_five_weather.csv"

# Créer un fichier CSV
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['City', 'Temperature (°C)', 'Weather']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df_top_five.iterrows():
        city = row['city']
        temperature = row['temperature']
        weather = row['weather']
        writer.writerow({'City': city, 'Temperature (°C)': temperature, 'Weather': weather})
        print(f"City: {city}, Temperature : {temperature}°C, Weather: {weather}")
   

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

City: Bormes les Mimosas, Temperature : 22°C, Weather: Clear
City: Aigues Mortes, Temperature : 21°C, Weather: Clear
City: Collioure, Temperature : 21°C, Weather: Clear
City: Cassis, Temperature : 20°C, Weather: Clear
City: Aix en Provence, Temperature : 19°C, Weather: Clear
City: Avignon, Temperature : 19°C, Weather: Clear
City: Nimes, Temperature : 18°C, Weather: Clear
City: Uzes, Temperature : 18°C, Weather: Clear
City: Carcassonne, Temperature : 17°C, Weather: Clear
City: Montauban, Temperature : 17°C, Weather: Clear
Les résultats ont été enregistrés dans 'meteo_data.csv'


Top 5 map

In [23]:
df_top_five.head()

,city,id,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Bormes les Mimosas,341,22,44,Clear,Hotel des Arts - Cite Bergere,https://www.booking.com/hotel/fr/des-arts-pari...,2 sur 5,L'Hotel des Arts est situé dans le centre de P...,48.871880,2.344655
1,Aigues Mortes,481,21,46,Clear,None,None,None,None,NaN,NaN
2,Collioure,502,21,62,Clear,Grand Hotel des Balcons,https://www.booking.com/hotel/fr/grand-des-bal...,2 sur 5,Situé dans le quartier chic de Saint-Germain à...,48.850422,2.339299
3,Cassis,361,20,42,Clear,LE SEPT charmant studio aux portes des calanques,https://www.booking.com/hotel/fr/le-sept-charm...,3 sur 5,Hébergement géré par un particulier,43.217600,5.532350
4,Aix en Provence,401,19,50,Clear,La Suite Albertas,https://www.booking.com/hotel/fr/le-555-mirabe...,3 sur 5,"Situé à Aix-en-Provence, à 100 mètres du cours...",43.527418,5.447695


In [24]:
colorscale = [
[0, 'rgb(57, 162, 225)'],
[1, 'rgb(234, 32, 41)']
]
fig = px.scatter_mapbox(df_top_five, lat="lat", lon="lon", color="temperature", size="humidity",
                         mapbox_style="open-street-map", zoom = 4, color_continuous_scale=colorscale, height=510, width=700)
fig.show()

Top 20 map

In [25]:
pd.set_option('display.max_rows', None)

In [26]:
df_top_20 =  df_final[df_final['city'].isin(df_top_five.loc[:,'city'].to_list())].loc[1:,['city', 'hotel', 'score', 'lat', 'lon', 'temperature']]
df_top_20.head(100)

,city,hotel,score,lat,lon,temperature
341,Bormes les Mimosas,Hotel des Arts - Cite Bergere,2 sur 5,48.87188026,2.34465466,22
342,Bormes les Mimosas,Hotel Audran,2 sur 5,48.88532693,2.33602367,22
343,Bormes les Mimosas,Hôtel De Venise,2 sur 5,48.84527191,2.38361210,22
344,Bormes les Mimosas,Hotel De La Cite Rougemont,2 sur 5,48.87209209,2.34528860,22
345,Bormes les Mimosas,Boutik Boheme L'Atelier Père Lachaise,1 sur 5,48.86668920,2.38539240,22
346,Bormes les Mimosas,Beautiful 1BR6P - Montorgueil RDC,1 sur 5,48.86676680,2.35161800,22
347,Bormes les Mimosas,District Nation,2 sur 5,48.85131070,2.39241260,22
348,Bormes les Mimosas,Hotel Diana Paris,2 sur 5,48.84989342,2.34510066,22
349,Bormes les Mimosas,HIGHSTAY - Luxury Serviced Apartments - Le Mar...,1 sur 5,48.86308680,2.35656140,22
350,Bormes les Mimosas,The Sleeping Lion,1 sur 5,48.85228200,2.36253100,22


In [27]:
df_top_20['score'] = df_top_20['score'].astype(str).str[0]
df_top_20['score'] = pd.to_numeric(df_top_20['score'], errors='coerce')
df_top_20.head()


,city,hotel,score,lat,lon,temperature
341,Bormes les Mimosas,Hotel des Arts - Cite Bergere,2.0,48.87188026,2.34465466,22
342,Bormes les Mimosas,Hotel Audran,2.0,48.88532693,2.33602367,22
343,Bormes les Mimosas,Hôtel De Venise,2.0,48.84527191,2.38361210,22
344,Bormes les Mimosas,Hotel De La Cite Rougemont,2.0,48.87209209,2.34528860,22
345,Bormes les Mimosas,Boutik Boheme L'Atelier Père Lachaise,1.0,48.86668920,2.38539240,22


In [28]:
df_top_20['lat'] = pd.to_numeric(df_top_20['lat'], errors='coerce')
df_top_20['lon'] = pd.to_numeric(df_top_20['lon'], errors='coerce')

In [29]:
fig = px.scatter_mapbox(df_top_20, lat="lat", lon="lon", color="score", mapbox_style="open-street-map", zoom = 10, size = "temperature", color_continuous_scale="oranges", 
                        hover_data={"city": True, "hotel": True, "lat": False, "lon": False}, height=510, width=700)
fig.show()